# Lee refined speckle filter evaluation

In [ ]:
%load_ext autoreload
%autoreload 2

import os


from polsarpro.dev.devtools import parse_psp_parameter_string
import numpy as np
from pathlib import Path
import xarray as xr
from polsarpro.io import read_C3_psp, read_T3_psp, open_netcdf_beam, polmat_to_netcdf
from polsarpro.speckle_filters import refined_lee
from polsarpro.util import S_to_C3, S_to_T3
from polsarpro.dev.metrics import summarize_metrics
import shutil

import matplotlib.pyplot as plt

# optional import for progress bar
from dask.diagnostics import ProgressBar

# change to your local C-PolSARpro install dir
c_psp_dir = "/home/c_psp/Soft/bin/"
os.environ["PATH"]+=os.pathsep+f"{c_psp_dir}/speckle_filter/"

# change to your data paths
# original dataset
input_alos_data = Path("/data/psp/test_files/SAN_FRANCISCO_ALOS1_slc.nc")

input_test_dir = Path("/data/psp/SAN_FRANCISCO_ALOS1")

# output files from C
output_test_dir = Path("/data/psp/res/ref_lee_c")
if not output_test_dir.exists():
    output_test_dir.mkdir(parents=True)

output_type = "T3"
if output_type not in ["T3", "C3"]:
    raise ValueError("output_type must be T3 or C3")

## Run the C-version on some test data

In [ ]:
input_str= f"""id: {input_test_dir} 
od: {output_test_dir}
iodf: S2{output_type}
nw: 7
nlk: 1
ofr: 0
ofc: 0
fnr: 18432
fnc: 1248
errf: /tmp/MemoryAllocError.txt
mask: {input_test_dir}/mask_valid_pixels.bin
"""
result = parse_psp_parameter_string(input_str)
os.system(f"lee_refined_filter.exe {result}")

# CPSP does not create another config file in output dir
shutil.copy(input_test_dir / "config.txt", output_test_dir)

## Load ALOS data and C outputs

In [ ]:
# uncomment to test on S matrix made with SNAP
S = open_netcdf_beam(input_alos_data)

## Apply the decomposition

In [ ]:
if os.path.exists(output_test_dir / "refined_lee_py.nc"):
    os.remove(output_test_dir / "refined_lee_py.nc")

with ProgressBar():
    if output_type == "T3":
        out_py = refined_lee(S_to_T3(S), window_size=7, num_looks=1)
    else:
        out_py = refined_lee(S_to_C3(S), window_size=7, num_looks=1)
    # use custom function that writes complex data and preserves chunks
    polmat_to_netcdf(out_py, output_test_dir / f"refined_lee_{output_type}_py.nc")
    

In [ ]:

if output_type == "T3":
    out_c = read_T3_psp(output_test_dir).compute()
else:
    out_c = read_C3_psp(output_test_dir).compute()
out_py = open_netcdf_beam(output_test_dir / f"refined_lee_{output_type}_py.nc").compute()

In [ ]:
df =summarize_metrics(out_c, out_py)
df